In [2]:
# import necessary lab
from src.graph.builder import umls_mapping_graph
import requests
import pandas as pd
import logging
from src.config.agents import AGENT_LLM_MAP
from src.prompts.template import apply_prompt_template

# Get more data for model performance evaluation

1.Get CUI code data through ‘HPO code’ and 'agent_code'

In [18]:
# Read the excel mapping results data
excel_file_path = "mapped_gc.xlsx"
df = pd.read_excel(excel_file_path)
print(df.head())

   Unnamed: 0             Section  \
0           0  Mother's Pregnancy   
1           1  Mother's Pregnancy   
2           2  Mother's Pregnancy   
3           3  Mother's Pregnancy   
4           4  Mother's Pregnancy   

                                            Question  \
0  Amniotic fluid issue - Too much fluid (polyhyd...   
1                           Decreased fetal movement   
2                              Delivery complication   
3                            Maternal health problem   
4                                    Placental issue   

                           true_term    HPO_code  true_CUI  \
0  Abnormality of the amniotic fluid  HP:0001560  C0266781   
1           Decreased fetal movement  HP:0001558  C0235659   
2                  Abnormal delivery  HP:0001787  C0549629   
3      Prenatal maternal abnormality  HP:0002686  C4531020   
4        Abnormality of the placenta  HP:0100767  C1306893   

                          agent_term  agent_code agent_CUI  confide

In [4]:
# getting CUI code from HPO code
def get_cui_from_ontology(hpo_code):
    """ Get CUI from a specific ontology term. """
    response = requests.get(f"{API_BASE_URL}/hpo_to_cui/{hpo_code}")
    print(f"Response content for HPO code {hpo_code}:", response.text)
    try:
        return response.json()["cui"]
    except KeyError:
        print("Key 'cui' not found in response.")
        return None  # or raise an error/log as needed


Get CUI for agent

In [7]:
API_BASE_URL = "http://localhost:8000/"  # UMLS API Base URL

df['true_CUI'] = None
df['agent_CUI'] = None
# Run the code
for idx, row in df.iterrows():
    hpo_code = row['HPO_code']
    agent_code = row['agent_code']
    
    # Get True CUI 
    true_cui = get_cui_from_ontology(hpo_code)
    df.at[idx, 'true_CUI'] = true_cui

    # Get agent_CUI 
    if hpo_code == agent_code:
        # If hpo_code = predicted hpo_code，then True_CUI = Predicted CUI
        df.at[idx, 'agent_CUI'] = true_cui
    else:
        agent_cui = get_cui_from_ontology(agent_code)
        df.at[idx, 'agent_CUI'] = agent_cui

# Check the results
print(df[['true_CUI', 'agent_CUI']].head())

Response content for HPO code HP:0001560: {"hpo_code":"HP:0001560","cui":"C0266781"}
Response content for HPO code HP:0001558: {"hpo_code":"HP:0001558","cui":"C0235659"}
Response content for HPO code HP:0001787: {"hpo_code":"HP:0001787","cui":"C0549629"}
Response content for HPO code HP:0002686: {"hpo_code":"HP:0002686","cui":"C4531020"}
Response content for HPO code HP:0100767: {"hpo_code":"HP:0100767","cui":"C1306893"}
Response content for HPO code HP:0011438: {"hpo_code":"HP:0011438","cui":"C4023357"}
Response content for HPO code HP:0001622: {"hpo_code":"HP:0001622","cui":"C0151526"}
Response content for HPO code HP:0010881: {"hpo_code":"HP:0010881","cui":"C0266785"}
Response content for HPO code HP:0001548: {"hpo_code":"HP:0001548","cui":"C1849265"}
Response content for HPO code HP:0001513: {"hpo_code":"HP:0001513","cui":"C0028754"}
Response content for HPO code HP:0012743: {"hpo_code":"HP:0012743","cui":"C0311277"}
Response content for HPO code HP:0001528: {"hpo_code":"HP:0001528

Get CUI for phenotagger

In [ ]:
# Run the code
for idx, row in df.iterrows():
    phetag_code = row['phetag_code']
    HPO_code = row['HPO_code']
    true_cui = row['true_CUI']

    # Get phetag_CUI 
    if phetag_code == HPO_code:
        # If phetag_code = HPO_code，then phetag_CUI = true_CUI
        df.at[idx, 'phetag_CUI'] = true_cui
    else:
        phetag_cui = get_cui_from_ontology(phetag_code)
        df.at[idx, 'phetag_CUI'] = phetag_cui

# Check the results
print(df[['true_CUI', 'phetag_CUI']].head())

Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0000824: {"hpo_code":"HP:0000824","cui":"C5539399"}
Response content for HPO code HP:0000824: {"hpo_code":"HP:0000824","cui":"C5539399"}
Response content for HPO code HP:0012125: {"hpo_code":"HP:0012125","cui":"C0376358"}
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0002664: {"hpo_code":"HP:0002664","cui":"C4732738"}
Response content for HPO code HP:0008069: {"h

Get CUI for phenoBERT

In [10]:
# Run the code
for idx, row in df.iterrows():
    BERT_code = row['BERT_code']
    HPO_code = row['HPO_code']
    true_cui = row['true_CUI']

    # Get BERT_CUI 
    if BERT_code == HPO_code:
        # If BERT_code = HPO_code，then BERT_CUI = true_CUI
        df.at[idx, 'BERT_CUI'] = true_cui
    else:
        BERT_cui = get_cui_from_ontology(BERT_code)
        df.at[idx, 'BERT_CUI'] = BERT_cui

# Check the results
print(df[['true_CUI', 'BERT_CUI']].head())

/var/folders/nb/4qmmwm7j43x3py5_nzg7b6w40000gn/T/ipykernel_28507/1783458182.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C0266781' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'BERT_CUI'] = true_cui


Response content for HPO code HP:0031437: {"hpo_code":"HP:0031437","cui":"C3544104"}
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0000824: {"hpo_code":"HP:0000824","cui":"C5539399"}
Response content for HPO code HP:0100651: {"hpo_code":"HP:0100651","cui":"C0011854"}
Response content for HPO code HP:0000824: {"hpo_code":"HP:0000824","cui":"C5539399"}
Response content for HPO code HP:0012125: {"hpo_code":"HP:0012125","cui":"C0376358"}
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0002890: {"hpo_code":"HP:0002890","cui":"C0549473"}
Response content for HPO code HP:0008069: {"hpo_code":"HP:0008069","cui":"C0037286"}
Response content for HPO code HP:003

Get CUI for GPT-5

In [13]:
# Run the code
for idx, row in df.iterrows():
    GPT5_code = row['GPT5_code']
    HPO_code = row['HPO_code']
    true_cui = row['true_CUI']

    # Get GPT5_CUI 
    if GPT5_code == HPO_code:
        # If GPT5_code = HPO_code，then GPT5_CUI = true_CUI
        df.at[idx, 'GPT5_CUI'] = true_cui
    else:
        GPT5_cui = get_cui_from_ontology(GPT5_code)
        df.at[idx, 'GPT5_CUI'] = GPT5_cui

# Check the results
print(df[['true_CUI', 'GPT5_CUI']].head())

/var/folders/nb/4qmmwm7j43x3py5_nzg7b6w40000gn/T/ipykernel_28507/317336054.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C0266781' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'GPT5_CUI'] = true_cui


Response content for HPO code HP:0001197: {"hpo_code":"HP:0001197","cui":"C4025797"}
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0011410: {"hpo_code":"HP:0011410","cui":"C0007876"}
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0001191: {"hpo_code":"HP:0001191","cui":"C1840535"}
Response content for HPO code HP:0000824: {"hpo_code":"HP:0000824","cui":"C5539399"}
Response content for HPO code nan: {"detail":"404: CUI not found for the given HPO code"}
Key 'cui' not found in response.
Response content for HPO code HP:0000819: {"hpo_code":"HP:0000819","cui":"C0011849"}
Response content for HPO code HP:0000824: {"hpo_code":"HP:0000824","cui":"C5539399"}
Response content for HPO code HP:0000823: {"hpo_code":"HP:0000823","cui":"C0034012"}
Response content for HPO code HP:000

In [14]:
# Output the result and check it 
df.to_excel("mapped_gc.xlsx")

2. Get the LLM score

In [16]:
# Define the calculate method
def rank_evaluate_with_llm(predicted_label, true_label, predicted_code, true_code, agent_llm_map):
    if is_null(predicted_code) or is_null(true_code):
        return None

    try:
        llm = agent_llm_map["rank_evaluate_with_llm"]
        prompt = apply_prompt_template(
    "rank_evaluate_with_llm", 
    {
        "predicted_label": predicted_label,
        "true_label": true_label,
        "predicted_code": predicted_code,
        "true_code": true_code
    }
)
        response = llm.invoke(prompt)

        if hasattr(response, "choices"):
            score_text = response.choices[0].message.content.strip()
        elif hasattr(response, "content"):
            score_text = response.content.strip()
        else:
            logging.error("LLM response structure not recognized.")
            return None

        try:
            score = float(score_text)
            if score < 1 or score > 10:
                logging.warning(f"LLM returned out-of-range score: {score_text}")
                return None
            return score

        except ValueError:
            logging.error(f"LLM returned invalid score: {score_text}")
            return None

    except Exception as e:
        logging.error(f"Error calling LLM agent for ranking evaluation: {e}")
        return None

Get LLM score for agent

In [ ]:
from functools import partial
# define is_null
def is_null(value):
    return pd.isna(value) or value is None or value == ''
    
evaluate_func = partial(rank_evaluate_with_llm, agent_llm_map=AGENT_LLM_MAP)

# Filter out rows without none values
filtered_df = df.dropna(subset=['true_CUI', 'agent_CUI']).query('true_CUI != agent_CUI')

# Apply the function
df['agent_LLM_Score'] = filtered_df.apply(
    lambda row: evaluate_func(
        row['agent_term'], 
        row['true_term'], 
        row['agent_CUI'], 
        row['true_CUI']
    ),
    axis=1
)

Get LLM score for phenotagger, phenoBERT, metamap, cTAKES, GPT-5

In [20]:
phetag_df = df.dropna(subset=['phetag_CUI', 'true_CUI']).query('true_CUI != phetag_CUI')
BERT_df = df.dropna(subset=['BERT_CUI', 'true_CUI']).query('true_CUI != BERT_CUI')
meta_df = df.dropna(subset=['meta_CUI', 'true_CUI']).query('true_CUI != meta_CUI')
# Apply the function
df['phetag_LLM_Score'] = df.apply(
    lambda row: evaluate_func(
        row['phetag_term'], 
        row['true_term'],
        row['phetag_CUI'], 
        row['true_CUI']
    ),
    axis=1
)
df['BERT_LLM_Score'] = df.apply(
    lambda row: evaluate_func(
        row['BERT_term'], 
        row['true_term'],
        row['BERT_CUI'], 
        row['true_CUI']
    ),
    axis=1
)
df['meta_LLM_Score'] = df.apply(
    lambda row: evaluate_func(
        row['meta_term'], 
        row['true_term'],
        row['meta_CUI'], 
        row['true_CUI']
    ),
    axis=1
)

In [ ]:
ctakes_df = df.dropna(subset=['ctakes_CUI', 'true_CUI']).query('true_CUI != ctakes_CUI')
df['ctakes_LLM_Score'] = df.apply(
    lambda row: evaluate_func(
        row['ctakes_term'], 
        row['true_term'],
        row['ctakes_CUI'], 
        row['true_CUI']
    ),
    axis=1
)

In [21]:
GPT5_df = df.dropna(subset=['GPT5_CUI', 'true_CUI']).query('true_CUI != GPT5_CUI')
df['GPT5_LLM_Score'] = df.apply(
    lambda row: evaluate_func(
        row['GPT5_term'], 
        row['true_term'],
        row['GPT5_CUI'], 
        row['true_CUI']
    ),
    axis=1
)

Output the results

In [1]:
# Output the results
df.to_csv("LLM score2.csv", index=False)

NameError: name 'df' is not defined